In [1]:
import numpy as np 
import pandas as pd

In [106]:
data = np.loadtxt("e_high_bonus.in",dtype=int)
rows,columns,n_vehicles,n_rides,bonus,steps = data[0,:]
actual_data = data[1:,:]
#row_start, column_start, row_end, column_end, earliest_start, latest finish
print(actual_data)

def distance(a,b,c,d):
    return abs(a-c) + abs(b-d)

distance(actual_data[0,0], actual_data[0,1], actual_data[0,2], actual_data[0,3])

all_distance = distance(actual_data[:,0], actual_data[:,1], actual_data[:,2], actual_data[:,3])
data_with_distances = np.hstack((actual_data, np.reshape(all_distance, (actual_data.shape[0],1))))
print(data_with_distances)
data_with_distances.shape

[[  295  1043   796  1955 38684 51050]
 [  434   467   200   833 89213 91692]
 [ 1473  1815  1321   810 17720 26914]
 ..., 
 [  938  1252  1076  1325   148  1289]
 [  133  1821    53    95 91282 96580]
 [  392  1993   493   790 86670 92380]]
[[  295  1043   796 ..., 38684 51050  1413]
 [  434   467   200 ..., 89213 91692   600]
 [ 1473  1815  1321 ..., 17720 26914  1157]
 ..., 
 [  938  1252  1076 ...,   148  1289   211]
 [  133  1821    53 ..., 91282 96580  1806]
 [  392  1993   493 ..., 86670 92380  1304]]


(10000, 7)

In [107]:
submit = {}
for i in range(n_vehicles):
    submit['F%s'%i] = [1,i]
    #if i > 32:
    #    submit['F%s'%i] = [0]

In [111]:
def calc_points(bonus, rides_data, submission_dict):
    assert isinstance(rides_data,(list,np.ndarray))
    assert isinstance(submission_dict,dict)
    dist_points = 0
    bonus_points = 0
    
    for f in submission_dict.keys():
        f_rides = submission_dict[f]
        assert len(f_rides) == f_rides[0] + 1, 'First number in list for each vehicle must equal the # of rides, occured for vehicle %s'%f
        
        f_position = np.array([0,0])
        t = 0
        for ride_num in f_rides[1:]:
            get_bonus = False
            start_row,start_col,end_row,end_col,earliest_start,latest_finish = rides_data[ride_num]
            distance = abs(end_row - start_row) + abs(end_col - start_col)
            #print(rides_data[ride_num])
            dist_to_start = np.sum(np.abs(f_position - np.array([start_row,start_col])))
            t += dist_to_start
            
            if t < earliest_start:
                #print('t less than earliest start %s, %s'%(t, earliest_start))
                bonus_points += bonus 
                get_bonus=True
            
            dist_points += distance
            t = max(t,earliest_start) + distance
            f_position = [end_row,end_col]
            
            if t > latest_finish:
                print('Warning, vehicle %s could not complete ride %s on time, finished at %s, should have finished at %s'%(f,ride_num,t,latest_finish))
                dist_points -= distance
                if get_bonus == True:
                    bonus_points -= bonus
            
    return dist_points + bonus_points
            
            
        
    print(bonus, rides_data, submission_dict)
calc_points(bonus, actual_data, submit)

766476

In [110]:
def submission_dict_to_file(submission_dict,output_file_name = 'out.txt'):
    assert isinstance(submission_dict,dict)
    file = ''
    for f in submission_dict.keys():
        f_rides = [str(int(x)) for x in submission_dict[f]]
        file += ' '.join(f_rides)
        file += '\n'
    text_file = open(output_file_name, "w")
    text_file.write(file)
    text_file.close()
submission_dict_to_file(submit,'e_out.txt')